# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve MNIST

In [1]:
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

Using TensorFlow backend.


### Sample problem: MNIST data set

The MNIST data set of handwritten gray scale images of digits 0-9 is a classic computer vision data set and therefore makes for good testing. Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
trainx = np.reshape(x_train,(60000,28,28,1))
testx = np.reshape(x_test, (10000,28,28,1))
trainy = keras.utils.np_utils.to_categorical(y_train, 10)
testy = keras.utils.np_utils.to_categorical(y_test, 10)
data = [trainx, trainy, testx, testy]

### Configuring NEAT

Many of the options and inputs are still handled through the config file. The config file has been shortened considerably as many parameters have been eliminated, although many parameters have also been introduced which could be added.

In [3]:
%%file configMNIST
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 28,28,1
output_nodes        = 10
conv                = True

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0
prob_addlayer         = 0.1
prob_addmodule        = 0.05
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coefficient       = 5.0
disjoint_coefficient     = 3.0
connection_coefficient   = 0.4
size_coefficient         = 0.8

[species]
species_size        = 5
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configMNIST


### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by CoDeepNEAT on NMIST and then use their accuracy after 10 epochs as their fitness.

In [4]:
def fitness(network, data):
    network.fit(data[0], data[1],  epochs=5)
    loss, acc = network.evaluate(data[2], data[3])
    return acc

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [5]:
def evolve(n, debugging=False):
    if(debugging):
        debug = open("debug.txt", "w")
    else:
        debug = None
    config.load('configMNIST')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(20, chromosome.ModuleChromo, debug=debug)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(5, chromosome.BlueprintChromo, module_pop, debug=debug)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 10, fitness, data, save_best=True, name='MNIST', debug=debug)
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="MNISTmod_")
    visualize.plot_stats(blueprint_pop.stats, name="MNISTbp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [6]:
evolve(5, True)

-----Generation 0--------
Network 0
Epoch 1/5
60000/60000 [==============================] - 9s 144us/step - loss: 14.5173 - acc: 0.0993
Epoch 2/5
60000/60000 [==============================] - 8s 136us/step - loss: 14.5176 - acc: 0.0993
Epoch 3/5
60000/60000 [==============================] - 9s 148us/step - loss: 14.5176 - acc: 0.0993
Epoch 4/5
60000/60000 [==============================] - 9s 156us/step - loss: 14.5176 - acc: 0.0993
Epoch 5/5
10000/10000 [==============================] - 1s 66us/step

Network 0 Fitness: 0.1032
Network 1
Epoch 1/5
60000/60000 [==============================] - 3s 55us/step - loss: 8.3185 - acc: 0.4771
Epoch 2/5
60000/60000 [==============================] - 3s 46us/step - loss: 7.4008 - acc: 0.5375
Epoch 3/5
60000/60000 [==============================] - 3s 45us/step - loss: 7.2573 - acc: 0.5474
Epoch 4/5
60000/60000 [==============================] - 3s 45us/step - loss: 7.1865 - acc: 0.5517
Epoch 5/5
10000/10000 [==============================] - 

60000/60000 [==============================] - 7s 112us/step - loss: 7.4592 - acc: 0.5359
Epoch 4/5
60000/60000 [==============================] - 7s 114us/step - loss: 7.3494 - acc: 0.5431
Epoch 5/5
10000/10000 [==============================] - 1s 54us/step

Network 2 Fitness: 0.5496
Network 3
Epoch 1/5
60000/60000 [==============================] - 5s 86us/step - loss: 9.9737 - acc: 0.3696
Epoch 2/5
60000/60000 [==============================] - 6s 96us/step - loss: 7.1700 - acc: 0.5466
Epoch 3/5
60000/60000 [==============================] - 5s 86us/step - loss: 6.2648 - acc: 0.6043
Epoch 4/5
60000/60000 [==============================] - 5s 77us/step - loss: 5.1612 - acc: 0.6722
Epoch 5/5
10000/10000 [==============================] - 1s 50us/step

Network 3 Fitness: 0.7907
Network 4
Epoch 1/5
60000/60000 [==============================] - 5s 84us/step - loss: 9.1943 - acc: 0.4132
Epoch 2/5
60000/60000 [==============================] - 5s 78us/step - loss: 4.9041 - acc: 0.6808
Ep

60000/60000 [==============================] - 6s 96us/step - loss: 9.4573 - acc: 0.4003
Epoch 2/5
60000/60000 [==============================] - 5s 83us/step - loss: 5.8222 - acc: 0.6283
Epoch 3/5
60000/60000 [==============================] - 5s 80us/step - loss: 3.9908 - acc: 0.7399
Epoch 4/5
60000/60000 [==============================] - 5s 82us/step - loss: 2.8215 - acc: 0.8011
Epoch 5/5
10000/10000 [==============================] - 1s 64us/step

Network 5 Fitness: 0.8936
Network 6
Epoch 1/5
60000/60000 [==============================] - 5s 85us/step - loss: 9.0403 - acc: 0.4279
Epoch 2/5
60000/60000 [==============================] - 4s 73us/step - loss: 5.3261 - acc: 0.6598
Epoch 3/5
60000/60000 [==============================] - 5s 78us/step - loss: 3.1995 - acc: 0.7930
Epoch 4/5
60000/60000 [==============================] - 4s 73us/step - loss: 2.7964 - acc: 0.8201
Epoch 5/5
10000/10000 [==============================] - 1s 63us/step

Network 6 Fitness: 0.8907
Network 7
Epoc

Epoch 5/5
10000/10000 [==============================] - 1s 80us/step

Network 7 Fitness: 0.2086
Network 8
Epoch 1/5
60000/60000 [==============================] - 6s 97us/step - loss: 8.4270 - acc: 0.4657
Epoch 2/5
60000/60000 [==============================] - 5s 80us/step - loss: 5.0364 - acc: 0.6777
Epoch 3/5
60000/60000 [==============================] - 5s 78us/step - loss: 4.2013 - acc: 0.7324
Epoch 4/5
60000/60000 [==============================] - 5s 77us/step - loss: 3.9523 - acc: 0.7491
Epoch 5/5
10000/10000 [==============================] - 1s 79us/step

Network 8 Fitness: 0.821
Network 9
Epoch 1/5
60000/60000 [==============================] - 6s 100us/step - loss: 14.2805 - acc: 0.1097
Epoch 2/5
60000/60000 [==============================] - 5s 82us/step - loss: 14.0586 - acc: 0.1245
Epoch 3/5
60000/60000 [==============================] - 5s 88us/step - loss: 13.8048 - acc: 0.1413
Epoch 4/5
60000/60000 [==============================] - 5s 84us/step - loss: 13.5722 - ac

In [7]:
def eval_best(model_file):
    config.load('configMNIST')
    model = keras.models.load_model(model_file)
    visualize.draw_net(model, "_" + model_file)    
    model.fit(data[0], data[1], epochs=5)
    loss, fitness = model.evaluate(data[2], data[3])
    print("fitness", fitness)

In [8]:
eval_best("MNIST_best_model_0")

Epoch 1/5
60000/60000 [==============================] - 5s 79us/step - loss: 1.1837 - acc: 0.9237
Epoch 2/5
60000/60000 [==============================] - 5s 77us/step - loss: 1.1270 - acc: 0.9277
Epoch 3/5
60000/60000 [==============================] - 5s 76us/step - loss: 1.0979 - acc: 0.9297
Epoch 4/5
60000/60000 [==============================] - 5s 77us/step - loss: 1.0422 - acc: 0.9335
Epoch 5/5
10000/10000 [==============================] - 1s 103us/step
fitness 0.9356
